# Hyperparameter Tuning with Keras Tuner for the DNN

We performed hyperparameter tuning to find the optimal number of neurons in each dense layer.

Best results: 320 for first layer and 224 for the second layer.

We then use this in the final DNN model.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    recall_score,
    roc_curve,
    roc_auc_score,
)
import shap

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.metrics import AUC
import keras_tuner as kt
from utils.data import load_and_preprocess_data
from sklearn.model_selection import GroupShuffleSplit
from utils.constants import NUMERICAL_FEATURES, RANDOM_SEED
from utils.plots import (
    plot_training_validation_auc,
    plot_training_validation_loss,
    plot_roc_curve,
)

## Pipeline Configuration

In [2]:
random_seed = RANDOM_SEED  # Random seed to ensure reproducibility
output_path = "output/tf_hyperparameter_tuning/"
cmap = "seismic"  # Colormap for SHAP plots use "seismic" for full cohort and "berlin" for biochem remission cohort
file_prefix = "tensorflow"
# file_prefix = "biochem_remission"

## Standardised Data Loading, Preprocessing and Splitting

In [3]:
df = load_and_preprocess_data() # pass biochem=True to use only biochem remission cohort

# Optimisation
columns_to_drop = [
    "has_active_symptoms",
    "baseline_eims_arthralgia_arthritis",
    "baseline_eims_ankylosing_spondylitis",
    "baseline_eims_erythema_nodosum",
    "baseline_eims_uveitis",
    "baseline_eims_scleritis_episclerities",
    "is_smoker_smokeryn1",
    "study_group_name_Await Dx",
    "ifx_drug_level",
    "ada_drug_level",
    "ifx_drug_level_present",
    "ada_drug_level_present",
    "ifx_antibody_present",
    "ada_antibody_present",
    "haematocrit",
]

df.drop(columns=columns_to_drop, inplace=True)

numerical_features = NUMERICAL_FEATURES
numerical_features = [col for col in numerical_features if col not in ['haematocrit', 'ada_drug_level', 'ifx_drug_level']]

In [ ]:
# Create Train Validate and Test Datasets

# First split into train and temp 70% train, 30% temp which will be split 50:50 into 15% val and 15% test

# GroupShuffleSplit
splitter = GroupShuffleSplit(test_size=0.36, n_splits=1, random_state=random_seed)

# Perform the split
for train_idx, test_idx in splitter.split(df, groups=df["study_id"]):
    train_data = df.iloc[train_idx]
    temp_data = df.iloc[test_idx]

# Drop 'study_id' from X_train and X_test as it's not a feature
X_train = train_data.drop(columns=["fatigue_outcome", "study_id"])
y_train = train_data["fatigue_outcome"]

groups = train_data["study_id"]  # Group variable for GroupKFold cross-validation

temp_data_splitter = GroupShuffleSplit(
    test_size=0.56, n_splits=1, random_state=random_seed
)

# Perform the split
for val_idx, test_idx in temp_data_splitter.split(
    temp_data, groups=temp_data["study_id"]
):
    val_data = df.iloc[val_idx]
    test_data = df.iloc[test_idx]

X_val = val_data.drop(columns=["fatigue_outcome", "study_id"])
y_val = val_data["fatigue_outcome"]

X_test = test_data.drop(columns=["fatigue_outcome", "study_id"])
y_test = test_data["fatigue_outcome"]


print("Train shape:", X_train.shape)
print("Val shape:", X_val.shape)
print("Test shape:", X_test.shape)
print("Y train shape:", y_train.shape)
print("Y val shape:", y_val.shape)
print("Y test shape:", y_test.shape)

In [5]:
# Normalize the numerical features using StandardScaler
X_unified = pd.concat([X_train, X_val, X_test])
unified_scaler = StandardScaler()
unified_scaler.fit(X_unified[numerical_features])

X_train[numerical_features] = unified_scaler.transform(X_train[numerical_features])
X_test[numerical_features] = unified_scaler.transform(X_test[numerical_features])
X_val[numerical_features] = unified_scaler.transform(X_val[numerical_features])


## Deep Learning with TensorFlow


In [6]:
def model_builder(hp):
    model = keras.Sequential()

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    number_of_layers = hp.Int("number_of_layers", min_value=1, max_value=2, step=1)
    for i in range(number_of_layers):
        hp_units = hp.Int("units_" + str(i), min_value=256, max_value=400, step=32)
        model.add(keras.layers.Dense(units=hp_units, activation="relu"))
        hp_dropout = hp.Float("dropout_" + str(i), min_value=0.2, max_value=0.4, step=0.1)
        model.add(keras.layers.Dropout(hp_dropout))

    model.add(keras.layers.Dense(1, activation="sigmoid"))

    model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=[AUC()])

    return model

In [7]:
tuner = kt.Hyperband(
    model_builder,
    objective="val_auc",
    max_epochs=50,
    factor=3,
    directory="working_data",
    project_name="keras_tuner",
)

In [8]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

In [ ]:
tuner.search(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping_callback],
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal hyperparameters are:
- Number of layers: {best_hps.get('number_of_layers')}
""")
for i in range(best_hps.get("number_of_layers")):
    print(f"Layer {i+1}: Units: {best_hps.get('units_' + str(i))}, Dropout: {best_hps.get('dropout_' + str(i))}")



### Model Definition

In [10]:
model = tuner.hypermodel.build(best_hps)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping_callback],
)

In [ ]:
model.summary()

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
plot_training_validation_auc(history_dict, output_path, file_prefix)

In [ ]:
plot_training_validation_loss(history_dict, output_path, file_prefix)

In [ ]:
results = model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred)

y_classes = np.where(y_pred > 0.5, 1, 0)

tn, fp, fn, tp = confusion_matrix(y_test, y_classes).ravel()

# Calculate metrics
accuracy = accuracy_score(y_test, y_classes)
sensitivity = recall_score(y_test, y_classes)  # TPR
specificity = tn / (tn + fp)  # TN

print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("AUC:", test_auc)

In [18]:
np.savetxt(output_path + "dnn_fpr.txt", fpr)
np.savetxt(output_path + "dnn_tpr.txt", tpr)

with open(output_path + "dnn_auc.txt", "w") as f:
    f.write(str(test_auc))

In [ ]:
plot_roc_curve(fpr, tpr, test_auc, output_path, file_prefix)